In [2]:
import warnings
warnings.filterwarnings('ignore')
from pandas import read_csv
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import boxplot, text
from tensorflow.keras.layers import Dense , Dropout,MaxPool2D
from tensorflow.keras.layers import LeakyReLU
import time
import math
import tensorflow as tf
from tensorflow.keras import backend as k
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from sklearn.metrics import r2_score
from tensorflow.keras import layers,models
from tensorflow.keras import callbacks
#from tensorflow.keras.utils.vis_utils import plot_model
import datetime
from tensorflow.keras.layers import Bidirectional

from tensorflow.keras.layers import LSTM
from keras import backend as k



In [4]:
## Load data
df_11=pd.read_csv(r"../input/siwads/1_3.csv",header=None)
df_22=pd.read_csv(r"../input/siwads/3_6.csv",header=None)
df_33=pd.read_csv(r"../input/siwads/6_9.csv",header=None)
df_44=pd.read_csv(r"../input/siwads/9_12.csv",header=None)
df_55=pd.read_csv(r"../input/siwads/12_15.csv",header=None)
df_66=pd.read_csv(r"../input/siwads/15_18.csv",header=None)
df_77=pd.read_csv(r"../input/siwads/18_20.csv",header=None)
frames = [df_11, df_22,df_33,df_44,df_55,df_66,df_77]
df = pd.concat(frames)



In [5]:
# rename columns for more readability
df.rename({0: 'H',
           1: 'Er',
           2: 'L',
           3: 'W',
           4: 'freq',
           5: 'amp_S11',
           6: 'phase_S11',
           7:'amp_S21',
           8:'phase_S21',}, axis=1, inplace=True)
#df_S21=pd.DataFrame(df, columns = ['Er', 'L', 'B','A','freq','amp_S21','phase_S21'])

In [6]:

dataset=df.values
##features
X=dataset[:,0:5]
#reshape into tensors
X = X.reshape(X.shape[0], X.shape[1], 1)
## targets
S_11 = dataset[:,5]
S_21=dataset[:,7]
phaseS11=dataset[:,6]
phaseS21=dataset[:,8]
# choose what targets u want to predict
Y=np.column_stack((S_11,S_21))
#Y=np.column_stack((phaseS11,phaseS21))

## split data

xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.20,random_state=42)
xtest_valid,xvalid,ytest_valid,yvalid= train_test_split(xtest, ytest, test_size=0.5,random_state=42)

print('trained data : ',round(len(dataset)*0.80))
print('tested and validation data :',(round(len(dataset)*0.2)))


trained data :  4416403
tested and validation data : 1104101


In [7]:
## I used a normalized mean square error
def NMSE (y_true, y_pred):
    nom = k.sum(k.square(k.abs(10**(y_true/20)-10**(y_pred/20))))
    denom = k.sum(k.square(k.abs(10**(y_true/20))))
    nmse = nom/denom
    #nmse1 = 10*k.log(nmse)
    return nmse

In [8]:
def NMSE_test (a, b):
    diff=[]
    denom1=[]
    for i in range(len(a)):
        diff.append(np.square(np.abs(10**(a[i]/20)-10**(b[i]/20))))
        denom1.append(np.square(np.abs(10**(a[i]/20))))
    nom = np.sum(diff)
    denom=np.sum(denom1)
    nmse = nom/denom
    #nmse1 = 10*k.log(nmse)
    return nmse

In [18]:
  early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                           patience = 20)

In [19]:
 
unit=70
tf.keras.backend.clear_session()
#Model declaration
model = Sequential()
model.add(Bidirectional(LSTM(units = unit,                             
         return_sequences=True),
         input_shape=(xtrain.shape[1], xtrain.shape[2])))
##model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units = unit,                             
         return_sequences=True)))
#model.add(Dropout(0.2))
#model.add(Bidirectional(LSTM(units = unit,return_sequences=True)))
model.add(Bidirectional(LSTM(units = unit)))
model.add(Dropout(0.2))
model.add(Dense(2))
#Compile model
model.compile(loss=NMSE, optimizer='adam', metrics=NMSE)
#Train the model
history = model.fit(xtrain, ytrain,validation_data=(xvalid,yvalid),callbacks=[early_stop],epochs=200,batch_size=5000, verbose=1)
#evaluate the model
print("evaluate = ",model.evaluate(xtrain, ytrain))
ypred = model.predict(xtest_valid)
#Global NMSE
NMSEG=NMSE_test(ytest_valid, ypred)
NMSElog=10*np.log10(NMSEG)
print("NMSE: %.4f" %(NMSEG) )
#NMSE the first target
NMSEG1=NMSE_test(ytest_valid[:,0], ypred[:,0])
NMSElog1=10*np.log10(NMSEG1)
#NMSE the second target
NMSEG2=NMSE_test(ytest_valid[:,1], ypred[:,1])
NMSElog2=10*np.log10(NMSEG2)
#save the metrics
file = open("LSTM_phase_3L_%s.txt"%(unit), "w")
 #convert variable to string
file.write("nmse = " +repr(NMSEG)+"/n"+"nmselog = " +repr(NMSElog)
          +"/n"+"nmse1 = " +repr(NMSEG1)+"/n"+"nmselog1 = " +repr(NMSElog1)
          +"/n"+"nmse2 = " +repr(NMSEG2)+"/n"+"nmselog2 = " +repr(NMSElog2))
#close file
file.close()
hist=history
#save the model
model.save("./LSTM_phase_3L_%s.h5"%(unit))
print("Saved model to disk")
#save the learning points
hist_df = pd.DataFrame(hist.history) 
hist_csv_file = './LSTM_phase_3L_%s.csv'%(unit)
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    

In [ ]:
#to plot the loss curve
df1=pd.read_csv('./LSTM_phase_3L_50.csv',header=None)
df1 = df1.iloc[1: , :]
df1.rename({0: 'epoch',
            1: 'loss',
            2: 'mse',
            3: 'val_loss',
            4: 'val_mse',}, axis=1, inplace=True)

df1 = df1.astype({'epoch':int,'mse':float,'val_mse':float,'loss':float,'val_loss':float})



plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
plt.plot(df1['mse'],color='b',label='NMSE')

plt.plot(df1['val_mse'],color='r',label='val_nmse')
plt.title('Training and Validation NMSE', fontsize=16)
plt.xlabel('Nbr of epoch')
plt.ylabel('NMSE')

plt.legend()
plt.grid()

plt.subplot(1,2,2)
plt.plot(df1['loss'],color='b',label='loss')


plt.plot(df1['val_loss'],color='r',label='val_loss')
plt.title('Training and Validation Loss', fontsize=16)
plt.xlabel('Nbr of epoch')
plt.ylabel('Loss')

plt.legend()
plt.grid()
plt.savefig('LSTM_phase_siw_train.png')
plt.show()
